In [1]:
import pandas as pd
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt 

import pandas as pd  
import seaborn as sns 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
drive.mount("/content/drive")

#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = pd.read_csv('Churn_Modelling.csv')

from tensorflow.python.client import device_lib
import tensorflow.test as test

test.gpu_device_name()
device_lib.list_local_devices()
!cat /proc/meminfo
tensorflow.device("gpu:0") 

MemTotal:       13333596 kB
MemFree:        10215212 kB
MemAvailable:   12348940 kB
Buffers:           83648 kB
Cached:          2163884 kB
SwapCached:            0 kB
Active:          1116624 kB
Inactive:        1701848 kB
Active(anon):     519256 kB
Inactive(anon):      360 kB
Active(file):     597368 kB
Inactive(file):  1701488 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               424 kB
Writeback:             0 kB
AnonPages:        570788 kB
Mapped:           373660 kB
Shmem:               992 kB
Slab:             194496 kB
SReclaimable:     151228 kB
SUnreclaim:        43268 kB
KernelStack:        5088 kB
PageTables:         7556 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666796 kB
Committed_AS:    3666440 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1024 kB
AnonHugePages:   

NameError: ignored

In [ ]:
from pandas_profiling import ProfileReport
%matplotlib inline

profile = ProfileReport(df, title="Churn", explorative=True)
profile.to_notebook_iframe()

In [ ]:
#Drop shit data
df=df.drop(columns=['Surname','RowNumber',"CustomerId"])

#Create X and Y
col=df.columns.to_list()
X1=df[col[:-2]].values
y1=df[col[-1]].values

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical


In [ ]:
categorical=['Geography','Gender']
df_worked=df
for name in categorical:
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(df[name])
   #print(integer_encoded)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    #print(onehot_encoded)

    for i in range (0 ,np.shape(onehot_encoded)[1]):

      df_worked["{} {} ".format(name, i)] = onehot_encoded[:,i]

    df_worked=df_worked.drop(columns=name)




from sklearn.preprocessing import LabelEncoder
labelendoder_X_1 = LabelEncoder()
X1[:, 1] = labelendoder_X_1.fit_transform(X1[:,1])
X1[:, 2] = labelendoder_X_1.fit_transform(X1[:,2])


from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np

ct = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
X1 = np.array(ct.fit_transform(X1), dtype=np.float)
df1=pd.DataFrame(X1)
df1


,0,1,2,3,4,5,6,7,8,9,10
0,1.0,0.0,0.0,619.0,0.0,42.0,2.0,0.00,1.0,1.0,1.0
1,0.0,0.0,1.0,608.0,0.0,41.0,1.0,83807.86,1.0,0.0,1.0
2,1.0,0.0,0.0,502.0,0.0,42.0,8.0,159660.80,3.0,1.0,0.0
3,1.0,0.0,0.0,699.0,0.0,39.0,1.0,0.00,2.0,0.0,0.0
4,0.0,0.0,1.0,850.0,0.0,43.0,2.0,125510.82,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.0,0.0,771.0,1.0,39.0,5.0,0.00,2.0,1.0,0.0
9996,1.0,0.0,0.0,516.0,1.0,35.0,10.0,57369.61,1.0,1.0,1.0
9997,1.0,0.0,0.0,709.0,0.0,36.0,7.0,0.00,1.0,0.0,1.0
9998,0.0,1.0,0.0,772.0,1.0,42.0,3.0,75075.31,2.0,1.0,0.0


In [ ]:
#Another way to HOT ENCODING
X_df = df
X_df = pd.concat([X_df,pd.get_dummies(X_df["Geograp"])])

In [ ]:
#dummy variable trap , if it is only 0 or 1 , remove one of them
df_worked=df_worked.drop(columns="Gender 1 ")

In [ ]:
from sklearn.preprocessing import StandardScaler

#Create X_test ,X_train
col_final = df_worked.columns.to_list()
target="Exited"
col_final.remove(target)
X=df_worked[col_final].values
y=df_worked[target].values

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=123)


X_train,X_test,y_train,y_test = train_test_split(df1.values,y1,test_size=0.3,random_state=123)

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test = sc.transform(X_test)

model=Sequential()
model.add(Dense(units=128,input_dim=13,activation='relu'))
model.add(Dense(units=64,activation='relu'))
model.add(Dense(units = 1 ,activation="relu"))

model.compile(loss="mse",optimizer='adam')



In [ ]:
print(model.summary())
print(X_test.shape)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               1792      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________
None
(3000, 12)


In [ ]:
model.fit(X_train, y_train, epochs=100)

Epoch 1/100


ValueError: ignored

In [ ]:
model.evaluate(X_test,y_test)

94/94 [==============================] - 0s 1ms/step - loss: 0.1107


0.11072023212909698

In [ ]:
from sklearn.preprocessing import StandardScaler

X_train,X_test,y_train,y_test = train_test_split(df1.values,y1,test_size=0.3,random_state=123)

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test = sc.transform(X_test)


#batch_size , 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

def create_model(new_opt):

  model=Sequential()
  model.add(Dense(units=6,input_dim=11,activation='relu'))
  model.add(Dense(units=6,activation='relu'))

  model.add(Dense(units =1 ,activation="sigmoid"))
  early_stopping = EarlyStopping(monitor="accuracy",patience=5)
  callbacks=[early_stopping,ModelCheckpoint(filepath='best_model.h5',monitor='accuracy',save_best_only=True)]
  model.compile(loss="BinaryCrossentropy",optimizer=new_opt,metrics=['accuracy'])

  return model , callbacks
#model2.fit(X_train, y_train, epochs=100)






In [ ]:
model2.evaluate(X_test,y_test)

94/94 [==============================] - 0s 806us/step - loss: 0.3254


0.3253801167011261

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

n_split = 10
accuracy_new = -100
accuracy = -10

search_opt =["adam" , "sgd"]
batch_opt = [10,20,30]
epochs_opt=[50,75,100]
for batch in batch_opt:
  for epoch in epochs_opt:

    for name in search_opt:
      ac=[]
      cv = KFold(n_split,random_state=666)
      if accuracy_new > accuracy:
          
          best_model = model
          accuracy = accuracy_new
      for train_index,test_index in cv.split(X1):
        x_train,x_test = X1[train_index] , X1[test_index]
        y_train,y_test = y1[train_index] , y1[test_index]
        model,callbacks = create_model(name)
        history=model.fit(x_train,y_train,epochs=epoch,
                          batch_size=batch,verbose=2,
                          callbacks=[callbacks])
        loss,accuracy_new = model.evaluate(x_test,y_test)
        ac.append(accuracy_new)
        accuracy_new = sum(ac)/len(ac)




/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Epoch 1/50
900/900 - 1s - loss: 66.2550 - accuracy: 0.7532
Epoch 2/50
900/900 - 1s - loss: 0.5223 - accuracy: 0.7961
Epoch 3/50
900/900 - 1s - loss: 0.5131 - accuracy: 0.7959
Epoch 4/50
900/900 - 1s - loss: 0.5049 - accuracy: 0.7960
Epoch 5/50
900/900 - 1s - loss: 0.5028 - accuracy: 0.7941
Epoch 6/50
900/900 - 1s - loss: 0.4997 - accuracy: 0.7941
Epoch 7/50
900/900 - 1s - loss: 0.4980 - accuracy: 0.7943
32/32 [==============================] - 0s 845us/step - loss: 0.4789 - accuracy: 0.7950
Epoch 1/50
900/900 - 1s - loss: 274.1142 - accuracy: 0.7293
Epoch 2/50
900/900 - 1s - loss: 7.1149 - accuracy: 0.7146
Epoch 3/50
900/900 - 1s - loss: 4.2366 - accuracy: 0.7173
Epoch 4/50
900/900 - 1s - loss: 4.9589 - accuracy: 0.7169
Epoch 5/50
900/900 - 1s - loss: 3.7773 - accuracy: 0.7203
Epoch 6/50
900/900 - 1s - loss: 3.1051 - accuracy: 0.7191
32/32 [==============================] - 0s 866us/step - loss: 0.4898 - accuracy: 0.7900
Epoch 1/50
900/900 - 1s - loss: 239.3533 - accuracy: 0.7331
Epoch

In [ ]:
loss,accuracy = model.evaluate(x_test,y_test)
accuracy

32/32 [==============================] - 0s 890us/step - loss: 12.8714 - accuracy: 0.4710


0.47099998593330383

In [ ]:
batch_size = (20,30)
epochs=(50,100)
optimizers ="adam" , "sgd"



In [ ]:
best_model.summary()

Model: "sequential_93"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_283 (Dense)            (None, 6)                 72        
_________________________________________________________________
dense_284 (Dense)            (None, 6)                 42        
_________________________________________________________________
dense_285 (Dense)            (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [ ]:
 cv = KFold(n_split,random_state=666).split(X1)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:
scores = cross_val_score(model, X1, y1, scoring='accuracy')


TypeError: ignored

In [ ]:
accuracy_new

0.7962999939918518

In [ ]:
ac=np.zeros((1))
np.append(ac,1)

array([0., 1.])

In [ ]:
np.mean(ac)

0.7962999939918518

In [ ]:
best_model.save('path/to/location') 

INFO:tensorflow:Assets written to: path/to/location/assets


In [ ]:
ac.append(1)

In [ ]:
sum(ac)/len(ac)

1.0

In [ ]:
ac

[-10, -10, -10, -10, -10, -10, -10, -10, -10, -10]